<a href="https://colab.research.google.com/github/BoHye0202/Toy-Project/blob/main/5.Music%20Recommendation%20based%20on%20emotion/Facial_Expression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Kaggle 데이터
https://www.kaggle.com/ananthu017/emotion-detection-fer

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"choibohye","key":"b82e31a50310da16da34248c963dfa0b"}'}

In [2]:
ls -1ha kaggle.json

kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        164           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        297           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        898            True  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       5519           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      44292            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge      11508            True  
connectx

In [4]:
!kaggle datasets download -d ananthu017/emotion-detection-fer

 98% 64.0M/65.2M [00:00<00:00, 79.5MB/s]
100% 65.2M/65.2M [00:00<00:00, 95.3MB/s]


In [5]:
!ls

emotion-detection-fer.zip  kaggle.json	sample_data


In [6]:
!unzip emotion-detection-fer.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/sad/im37.png      
  inflating: train/sad/im370.png     
  inflating: train/sad/im3700.png    
  inflating: train/sad/im3701.png    
  inflating: train/sad/im3702.png    
  inflating: train/sad/im3703.png    
  inflating: train/sad/im3704.png    
  inflating: train/sad/im3705.png    
  inflating: train/sad/im3706.png    
  inflating: train/sad/im3707.png    
  inflating: train/sad/im3708.png    
  inflating: train/sad/im3709.png    
  inflating: train/sad/im371.png     
  inflating: train/sad/im3710.png    
  inflating: train/sad/im3711.png    
  inflating: train/sad/im3712.png    
  inflating: train/sad/im3713.png    
  inflating: train/sad/im3714.png    
  inflating: train/sad/im3715.png    
  inflating: train/sad/im3716.png    
  inflating: train/sad/im3717.png    
  inflating: train/sad/im3718.png    
  inflating: train/sad/im3719.png    
  inflating: train/sad/im372.png     
  inflating: train/sad/im3720.png    
  inflating: t

# 2. 데이터 불러오기

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [11]:
path = '/content/train/'

os.listdir(path)

['neutral', 'sad', 'angry', 'happy', 'fearful', 'surprised', 'disgusted']